Loading libraries and Original Dataset<br />
Dataset Source: <a href="https://www.kaggle.com/datasets/arshkon/linkedin-job-postings/data">LinkedIn Job Postings (2023 - 2024)</a>


In [1]:
# importing basic libraries
import pandas as pd
import math
import os
# importing local file which contains a dictionnary
from state_dict import *

# Dataset sources folder 
path_to_file = '../Datasets/Linkedin_Job_Posting/'

In [2]:
# Setting up stuff for pandas reading
# Displaying all columns
pd.set_option('display.max_columns', 40)

In [3]:
# Reading the Jobposting CSV
dataset = pd.read_csv(path_to_file+'postings.csv')

In [ ]:
# Checking dataset columns names
print(dataset.columns)

In [6]:
# Extracting the list of States in the location field and adding it to the dataset
all_locations = dataset['location'].tolist()
# Adding a new column in our dataset to store the State
dataset['state'] = str(0)

# Temp var
temp_states = []

# We need to split the data to only keep the States
# Some entries are in the form "city, State", some are just generic text of city name (or "United State")
for i in all_locations:
    split_loc = [x.strip() for x in i.split(",")]
    if len(split_loc) > 1:
        # We have a lot of State with a loc in the form of "STATE Metropolitan Area" so we remove those parts
        state = split_loc[1].replace(' Area', '')
        state = state.replace(' Metropolitan', '')

        # If the result is 'United States', we check if the first element is in our State Dictionnary, OTherwise we store "United States"
        if state == 'United States':
            if split_loc[0] in state_conversion:
                temp_states.append(state_conversion[split_loc[0]])
            else:
                temp_states.append(state)
        # We check if we already have a two-digits letter name, we keep it as is
        elif len(state) == 2:
            temp_states.append(state)
        # Otherwise, we convert the full name to a two-digits letter name
        elif state in state_conversion:
            temp_states.append(state_conversion[state])
        # Almost no occurences - we store "Other"
        else:
            temp_states.append('Other')
    else:
        # We keep the "United States" value but override the rest with Other as it's about 4% of the full dataset
        if i.strip() == 'United States':
            temp_states.append('United States')
        else:
            temp_states.append('Other')

# temp_states

# Storing the values in our dataset
# TODO : optimize this as it takes about 10 seconds to just copy data
for i in range(len(temp_states)):
    dataset.loc[i, 'state'] = temp_states[i]

First File is About all "Data" Offers


In [7]:
# We filter all the data to only keep job titles containing the word "Data" 
mask = dataset['title'].str.contains('data', case=False)
subdf_jobs = dataset[mask]

In [8]:
# We drop useless columns

# We have two entries, one in comments, because I modified the original CSV to remove a column from there
# The column was responsible for over 450Mo of data and prevented us to easily upload the file to github

# columns_to_drop = ['views', 'formatted_work_type', 'applies', 'original_listed_time', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'compensation_type', 'max_salary', 'pay_period', 'med_salary', 'min_salary', 'currency', 'description']
columns_to_drop = ['views', 'formatted_work_type', 'applies', 'original_listed_time', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'compensation_type', 'max_salary', 'pay_period', 'med_salary', 'min_salary', 'currency']
subdf_jobs_cleaned = subdf_jobs.drop(labels = columns_to_drop, axis=1)

In [9]:
# We write the sub df content into a new CSV file
filename = 'all_offers_data.csv'

#deleting file if a version already exists
if filename in os.listdir("./csv"):
    os.remove('./csv/'+filename)

# Writing the CSV
subdf_jobs_cleaned.to_csv('./csv/'+filename, sep=',', na_rep='N/A')

Second One is about the ratio of Remote options per States


In [11]:
# Creating a new Dataset where we will save our data
# Data will be stats per Location (States, mostly) about the share of remote work
# We will need the State, but also data on number of offer in that state, number with remote option, and a share of remote (which is remote/total)
# We also calculate Quartiles so we can split the data into groups for the Data Viz part

subdf_remote = pd.DataFrame()
subdf_remote['state'] = str(0)
subdf_remote['total_offer'] = 0
subdf_remote['remote_offer'] = 0
subdf_remote['remote_share'] = 0
subdf_remote['remote_quartile'] = 0

In [12]:
# We replace all NaN values in the dataset['remote_allowed'] column by 0, as it works as a True/False and save it in a new dataset 
# df1 is an intermediate DF for cleaning data and stuff
df1 = pd.DataFrame()
df1['remote_allowed'] = dataset['remote_allowed']
df1.fillna({'remote_allowed':0}, inplace=True)
df1['state'] = dataset['state']


In [31]:
# We look for the total number of offers per states and populate our main DataFrame
subset = df1.groupby('state', as_index=False).count()
subdf_remote['state'] = subset['state']
subdf_remote['total_offer'] = subset['remote_allowed']

#Now we make a sum to only get the number of remote offers per state
subset = df1.groupby('state', as_index=False).sum()
subdf_remote['remote_offer'] = subset['remote_allowed'].astype('int64')

# We calculate the share of remote work for each State
for i in range(len(subdf_remote)):
    subdf_remote.loc[i, 'remote_share'] = round(subdf_remote.loc[i, 'remote_offer']/subdf_remote.loc[i, 'total_offer']*100, 2)

# We calculate the quartile each State belongs to based on its share of remote work and set a score based on the quartile 
q1, q2, q3 = subdf_remote['remote_share'].quantile([0.25,0.5,0.75])

temp_qtl = []

for i in subdf_remote['remote_share']:
    if i < q1:
        temp_qtl.append(1)
    elif i < q2:
        temp_qtl.append(2)
    elif i < q3:
        temp_qtl.append(3)
    else:
        temp_qtl.append(4)

for i in range(len(temp_qtl)):
    subdf_remote.loc[i, 'remote_quartile'] = temp_qtl[i]




[4, 3, 2, 3, 4, 4, 4, 4, 1, 3, 3, 1, 2, 3, 4, 2, 2, 1, 1, 4, 3, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 4, 1, 2, 3, 3, 2, 1, 3, 4, 3, 1, 4, 1, 2, 3, 3, 3, 4, 4, 4, 4, 3, 2, 2]


In [14]:
# We make another csv with this data
filename = 'remote_work_share.csv'

#deleting file if a version already exists
if filename in os.listdir("./csv"):
    os.remove('./csv/'+filename)

# Writing the CSV
subdf_remote.to_csv('./csv/'+filename, sep=',', na_rep='N/A')